In [10]:
import numpy as np
import sklearn.datasets
import cvxopt

Import the dataset files:

In [11]:
x_train, y_train = sklearn.datasets.load_svmlight_file('./dataset/satimage.scale.tr')
x_val, y_val = sklearn.datasets.load_svmlight_file('./dataset/satimage.scale.val')
x_test, test_y = sklearn.datasets.load_svmlight_file('./dataset/satimage.scale.t')

Convert sparse matrices to dense matrices:

In [12]:
x_train = np.asarray(x_train.todense())
print(x_train)
x_val = x_val.todense()
x_test = x_test.todense()

[[ 0.          0.127273   -0.0952381  ... -0.126214   -0.431579
  -0.484375  ]
 [ 0.         -0.0909091  -0.571429   ... -0.126214   -0.494737
  -0.609375  ]
 [ 0.5625      0.490909    0.333333   ...  0.00970874 -0.326316
  -0.4375    ]
 ...
 [ 0.78125     0.545455    0.571429   ...  0.417476    0.136842
  -0.0625    ]
 [ 0.3125      0.254545    0.142857   ... -0.223301   -0.0947368
  -0.203125  ]
 [ 0.125       0.0545455  -0.190476   ...  0.0873786  -0.178947
  -0.34375   ]]


Separate data points that belong to class 4 or 6.

In [13]:
x_train = x_train[np.in1d(y_train, (4, 6))]
y_train = y_train[np.in1d(y_train, (4, 6))]
y_train = y_train.replace

# x_train = x_train[(y_train == 4) | (y_train == 6)]
# y_train = y_train[(y_train == 4) | (y_train == 6)]

In [14]:
K = (y_train.reshape(-1, 1) * x_train)
# print(x_train)
# print(y_train)
# print(k)

qc = K @ K.T
# print(qc)

In [15]:
n = x_train.shape[0]
C = 1

P = cvxopt.matrix(qc.T)  # FIXME: To transpose or not to transpose...
q = cvxopt.matrix(-1 * np.ones(n))
G = cvxopt.matrix(np.concatenate((-1 * np.identity(n), np.identity(n)), axis=0))
# G = cvxopt.matrix(np.concatenate((-1*np.identity(n_samples), np.identity(n_samples)), axis=0))

h = cvxopt.matrix(np.concatenate((-1 * np.zeros(n), C * np.ones(n)), axis=0))
A = cvxopt.matrix(1.0 * y_train, (1, n))
b = cvxopt.matrix(0.0)
for kooft in (P, q, G, h, A, b):
    print(f'{kooft.size}')

cvxopt.solvers.options['show_progress'] = False
solution = cvxopt.solvers.qp(P, q, G, h, A, b)

alpha = np.ravel(solution['x']).reshape(-1, 1)

S = np.where((alpha > 1e-10) & (alpha <= C))[0]
w = K.T[:, S] @ alpha[S]

print(w.shape)

M = np.where((alpha > 1e-10) & (alpha < C))[0]
b = np.mean(y_train[M] - x_train[M, :] @ w)

(847, 847)
(847, 1)
(1694, 847)
(1694, 1)
(1, 847)
(1, 1)
(36, 1)


In [16]:

# import cvxopt
# import numpy as np

# # from .base import BaseModel


# class SVMSoftMargin():

#     def __init__(self, C: float):
#         self.C = C
#         self._w = None
#         self._b = None

#     def train(self, X: np.ndarray, y: np.ndarray):
#         n_samples, _ = X.shape
#         # compute inputs for cvxopt solver
#         K = (X * y[:, np.newaxis]).T
#         P = cvxopt.matrix(K.T.dot(K)) # P has shape n*n
#         q = cvxopt.matrix(-1 * np.ones(n_samples)) # q has shape n*1
#         G = cvxopt.matrix(np.concatenate((-1*np.identity(n_samples), np.identity(n_samples)), axis=0))
#         h = cvxopt.matrix(np.concatenate((np.zeros(n_samples), self.C*np.ones(n_samples)), axis=0))
#         A = cvxopt.matrix(1.0 * y, (1, n_samples))
#         b = cvxopt.matrix(0.0)
#         # solve quadratic programming
#         for kooft in (P, q, G, h, A, b):
#             print(f'{kooft.size})')
#         cvxopt.solvers.options['show_progress'] = False
#         solution = cvxopt.solvers.qp(P, q, G, h, A, b)
#         _lambda = np.ravel(solution['x'])
#         # find support vectors
#         S = np.where((_lambda > 1e-10) & (_lambda <= self.C))[0]
#         self._w = K[:, S].dot(_lambda[S])
#         M = np.where((_lambda > 1e-10) & (_lambda < self.C))[0]
#         self._b = np.mean(y[M] - X[M, :].dot(self._w))

#     def predict(self, X: np.ndarray) -> np.ndarray:
#         """
#         Return +1 for positive class and -1 for negative class.
#         """
#         results = np.sign(X.dot(self._w) + self._b)
#         results[results == 0] = 1
#         return results

In [17]:
svm = SVMSoftMargin(C=1e-5)
svm.train(x_train, y_train)


NameError: name 'SVMSoftMargin' is not defined